In [0]:
%sql
select * from dev.spark_db.flight_time limit 2

In [0]:
flight_time_df = spark.read.table("dev.spark_db.flight_time")

flight_time_df_1 = (
    flight_time_df.selectExpr("fl_date as dep_date",
                              "to_date(dep_date+dep_time+wheels_on+taxi_in)as arr_date",
                              "dep_date + crs_dep_time as crs_dep_time",
                              "DEP_TIME + DEP_TIME as dep_time",
                              "arr_date + CRS_ARR_TIME as crs_arr_time",
                              "arr_date + arr_time as arr_time"
                              )
)

flight_time_df_1.where("OP_CARRIER_FL_NUM =1451 ").display()

In [0]:
from pyspark.sql.functions import expr
flight_time_df_2 = (
    flight_time_df.withColumnRenamed("fl_date","dep_date")
    .withColumn("arr_date",expr("dep_date+dep_time+wheels_on+taxi_in"))
    .withColumns({
            "dep_time":expr("dep_date + crs_dep_time"),
            "dep_time":expr("DEP_TIME + DEP_TIME"),
            "crs_arr_time":expr("arr_date + CRS_ARR_TIME"),
             "arr_time":expr("arr_date + arr_time")
    })

)
flight_time_df_2.limit(3).display()

In [0]:
from pyspark.sql.functions import col,to_date

flight_time_df_2 = (
    flight_time_df.withColumnRenamed("fl_date","dep_date")
    .withColumn("arr_date",to_date(col("dep_date")+col("dep_time")+col("wheels_on")+col("taxi_in")))
    .withColumns({
            "dep_time":col("dep_date") + col("crs_dep_time"),
            "dep_time":col("DEP_TIME") + col("DEP_TIME"),
            "crs_arr_time":col("arr_date") + col("CRS_ARR_TIME"),
             "arr_time":col("arr_date") + col("arr_time")
    })

)
flight_time_df_2.where( (col("OP_CARRIER_FL_NUM")==1451) & (col("dep_date")=='2000-01-01')).display()